# <span style="color:orange">30/05 - Pair Test estadisticos 34 Alicia y Lidia</span>

In [27]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy.stats import skew 
from scipy.stats import kurtosistest 
from scipy import stats
from scipy.stats import kstest 
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import math

import researchpy as rp
import scipy.stats as stats
from scipy.stats import levene


import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

In [28]:
df = pd.read_csv("./listing_nuevo.csv", index_col=0)
df.head(2)

,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,59,1,81,1,149,3
1,Latina,Private room,31,4,33,2,131,0


### 1. **Estandarizar las columnas de las variables predictoras**

In [29]:
df_estandarizar = df.drop(["room_type","neighbourhood_group"], axis=1)

In [30]:
st = StandardScaler()
df_std = pd.DataFrame(st.fit_transform(df_estandarizar), columns=df_estandarizar.columns)
df_std.head()

,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,-0.399193,-0.177300,0.647626,-0.381540,0.006611,-0.269176
1,-0.617321,-0.094826,-0.048250,-0.345623,-0.122269,-0.493611
2,-0.469305,0.207580,-0.526665,-0.201955,1.166533,-0.493611
3,-0.142114,-0.067334,-0.323701,-0.381540,0.135492,-0.194364
4,-0.632901,-0.149809,1.705938,-0.381540,1.402814,-0.119552


In [31]:
columnas = df_std.columns
df[columnas] = df_std
df.head()

,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,-0.399193,-0.177300,0.647626,-0.381540,0.006611,-0.269176
1,Latina,Private room,-0.617321,-0.094826,-0.048250,-0.345623,-0.122269,-0.493611
2,Arganzuela,Entire home/apt,-0.469305,0.207580,-0.526665,-0.201955,1.166533,-0.493611
3,Centro,Entire home/apt,-0.142114,-0.067334,-0.323701,-0.381540,0.135492,-0.194364
4,Arganzuela,Private room,-0.632901,-0.149809,1.705938,-0.381540,1.402814,-0.119552


### 2. **Cread un ANOVA, siendo la variable respuesta price y las variables predictoras el resto de las columnas del dataframe**

In [32]:
lm = ols('price ~ neighbourhood_group  + room_type + minimum_nights + number_of_reviews + calculated_host_listings_count + availability_365 + number_of_reviews_ltm', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
neighbourhood_group,20.0,121.197115,6.059856,6.449652,7.186450e-18
room_type,3.0,1.957825,0.652608,0.694587,5.552183e-01
minimum_nights,1.0,1.919105,1.919105,2.042550,1.529710e-01
number_of_reviews,1.0,158.991393,158.991393,169.218414,1.659438e-38
calculated_host_listings_count,1.0,24.951687,24.951687,26.556688,2.587226e-07
availability_365,1.0,350.728883,350.728883,373.289296,2.630761e-82
number_of_reviews_ltm,1.0,48.197376,48.197376,51.297642,8.255189e-13
Residual,17355.0,16306.119211,0.939563,NaN,NaN


**🤓  ¿Que podemos ver en resumen ANOVA? 👇🏻**

**df = grado de libertad/ Nº observaciones en los datos que pueden varian libremente.**

**"neighbourhood_group" y "room_type" Nº de valores unicos -1 dentro de la columna.**

**Sí es numérica siempre será =1, solo varia dentro de un eje.**



**F= "availability_365" es la que mayor capacidad explicativa tiene la variable predictora sobre la variación de la variable respuesta.**



**PR(>F) = "availability_365" tenemos en cuenta que es el valor más pequeño(pvalor<0,05), rechazamos la HO, la disponibilidad si influye sobre el precio de la casa.**

**Podemos observar que "room_type"  y "minimum_nights" tienen un pvalor > 0,05, en este caso deberíamos eliminar estas variables.**


In [33]:
media = df["price"].mean()
media

-0.006508714921081153

In [34]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     26.91
Date:                Mon, 30 May 2022   Prob (F-statistic):          2.04e-137
Time:                        15:58:50   Log-Likelihood:                -24110.
No. Observations:               17384   AIC:                         4.828e+04
Df Residuals:                   17355   BIC:                         4.850e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       -0.0301      0.032     -0.941      0.347      -0.093       0.033
neighbourhood_group[T.Barajas]                   0.0317      0.089      0.358      0.721      -0.142       0.205
neighbourhood_group[T.Carabanchel]               0.0875      0.051      1.733      0.083      -0.011       0.187
neighbourhood_group[T.Centro]                   -0.0221      0.033     -0.666      0.506      -0.087       0.043
neighbourhood_group[T.Chamartín]                 0.1405      0.054      2.624      0.009       0.036       0.245
neighbourhood_group[T.Chamberí]                  0.0148      0.043      0.342      0.732      -0.070       0.100
neighbourhood_group[T.Ciudad Lineal]             0.1303      0.053      2.458      0.014       0.026       0.234
neighbourhood_group[T.Fuencarral - El Pardo]     0.1083      0.067      1.616      0.106      -0.023       0.240
neighbourhood_group[T.Hortaleza]                 0.0951      0.061      1.562      0.118      -0.024       0.214
neighbourhood_group[T.Latina]                    0.0563      0.054      1.038      0.299      -0.050       0.163
neighbourhood_group[T.Moncloa - Aravaca]        -0.0332      0.054     -0.618      0.537      -0.139       0.072
neighbourhood_group[T.Moratalaz]                 0.1663      0.095      1.755      0.079      -0.019       0.352
neighbourhood_group[T.Puente de Vallecas]        0.0188      0.055      0.345      0.730      -0.088       0.126
neighbourhood_group[T.Retiro]                   -0.0311      0.050     -0.618      0.537      -0.130       0.068
neighbourhood_group[T.Salamanca]                -0.0222      0.042     -0.527      0.598      -0.105       0.060
neighbourhood_group[T.San Blas - Canillejas]     0.4119      0.057      7.208      0.000       0.300       0.524
neighbourhood_group[T.Tetuán]                    0.0151      0.047      0.319      0.750      -0.078       0.108
neighbourhood_group[T.Usera]                    -0.0409      0.069     -0.593      0.553      -0.176       0.094
neighbourhood_group[T.Vicálvaro]                 0.1258      0.122      1.033      0.302      -0.113       0.365
neighbourhood_group[T.Villa de Vallecas]        -0.0888      0.115     -0.773      0.440      -0.314       0.136
neighbourhood_group[T.Villaverde]               -0.0517      0.082     -0.628      0.530      -0.213       0.110
room_type[T.Hotel room]                          0.0379      0.081      0.467      0.640      -0.121       0.197
room_type[T.Private room]                        0.0233      0.016      1.447      0.148      -0.008       0.055
room_type[T.Shared room]                         0.0030      0.065      0.047      0.963      -0.123       0.130
minimum_nights                  

**🤓  ¿Que podemos ver en summary? 👇🏻**

...

**coef= la mayoria de nuestros coeficientes es positivo.el coef representa los cambios medios en la variable respuesta para una unidad de cambio en la varibale predictora, dejando las demás constantes. En nuestro DF la mayoría tienen relación positiva por lo que el coef aumenta. Tendremos que fijarnos ahora en el error apra saber si esa estimaición es precisa o no.**

**t= podemos verificar que coeficien es el que más seguridad en su predicción tiene.**

**Podemos resaltar en este caso, que las mejores prediciones las encontramos en "neighbourhood_group"[T.San Blas - Canillejas], "number_of_reviews", "calculated_host_listings_count", "availability_365" y "number_of_reviews_ltm", son aquellas con un pvalor < 0,05, (en este caso todas ellas tienen 0,0), y una t bastante alta en comparación al resto.**

**R-squared: es igual a 0.042 quiere decir que este modelo de precicción cumple su cometido en un 4,2% un número muy bajo**




------

**🤔 Eliminamos aquellas columnas que nos han dado un pvalor> 0,05 para ver si puede mejorar el modelo.**

In [35]:
# ANOVA SIN LAS VARIABLES  "room_type"  y "minimum_nights" 
lm = ols('price ~ neighbourhood_group + number_of_reviews + calculated_host_listings_count + availability_365 + number_of_reviews_ltm', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
neighbourhood_group,20.0,121.197115,6.059856,6.447085,7.345940e-18
number_of_reviews,1.0,157.976821,157.976821,168.071645,2.937560e-38
calculated_host_listings_count,1.0,25.279979,25.279979,26.895387,2.171833e-07
availability_365,1.0,347.222612,347.222612,369.410369,1.765025e-81
number_of_reviews_ltm,1.0,46.013206,46.013206,48.953481,2.716710e-12
Residual,17359.0,16316.372863,0.939937,NaN,NaN


**🤓  ¿Que podemos ver en resumen ANOVA? 👇🏻**

**Ahora todas la variables tienen un pvalor < 0,05**

**Todos los valores han variado en muy poca medida**

**PR(>F) = "availability_365" tenemos en cuenta que es el valor más pequeño(pvalor<0,05),ha variado del anterior, pero seguimos rechazando la HO, la disponibilidad si influye sobre el precio de la casa.**



**Ahora todas la variables tienen un pvalor < 0,05**

In [36]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     30.93
Date:                Mon, 30 May 2022   Prob (F-statistic):          4.45e-138
Time:                        15:58:51   Log-Likelihood:                -24116.
No. Observations:               17384   AIC:                         4.828e+04
Df Residuals:                   17359   BIC:                         4.848e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       -0.0200      0.031     -0.641      0.522      -0.081       0.041
neighbourhood_group[T.Barajas]                   0.0346      0.089      0.391      0.696      -0.139       0.208
neighbourhood_group[T.Carabanchel]               0.0921      0.050      1.827      0.068      -0.007       0.191
neighbourhood_group[T.Centro]                   -0.0258      0.033     -0.779      0.436      -0.091       0.039
neighbourhood_group[T.Chamartín]                 0.1404      0.054      2.621      0.009       0.035       0.245
neighbourhood_group[T.Chamberí]                  0.0141      0.043      0.326      0.745      -0.071       0.099
neighbourhood_group[T.Ciudad Lineal]             0.1352      0.053      2.556      0.011       0.032       0.239
neighbourhood_group[T.Fuencarral - El Pardo]     0.1133      0.067      1.692      0.091      -0.018       0.245
neighbourhood_group[T.Hortaleza]                 0.0977      0.061      1.605      0.109      -0.022       0.217
neighbourhood_group[T.Latina]                    0.0599      0.054      1.107      0.268      -0.046       0.166
neighbourhood_group[T.Moncloa - Aravaca]        -0.0317      0.054     -0.590      0.556      -0.137       0.074
neighbourhood_group[T.Moratalaz]                 0.1747      0.095      1.847      0.065      -0.011       0.360
neighbourhood_group[T.Puente de Vallecas]        0.0222      0.054      0.407      0.684      -0.085       0.129
neighbourhood_group[T.Retiro]                   -0.0328      0.050     -0.651      0.515      -0.132       0.066
neighbourhood_group[T.Salamanca]                -0.0260      0.042     -0.617      0.537      -0.108       0.057
neighbourhood_group[T.San Blas - Canillejas]     0.4169      0.057      7.315      0.000       0.305       0.529
neighbourhood_group[T.Tetuán]                    0.0143      0.047      0.302      0.763      -0.079       0.107
neighbourhood_group[T.Usera]                    -0.0346      0.069     -0.502      0.616      -0.170       0.101
neighbourhood_group[T.Vicálvaro]                 0.1277      0.122      1.050      0.294      -0.111       0.366
neighbourhood_group[T.Villa de Vallecas]        -0.0829      0.115     -0.722      0.470      -0.308       0.142
neighbourhood_group[T.Villaverde]               -0.0501      0.082     -0.610      0.542      -0.211       0.111
number_of_reviews                               -0.0667      0.009     -7.514      0.000      -0.084      -0.049
calculated_host_listings_count                   0.0320      0.007      4.339      0.000       0.018       0.046
availability_365                                 0.1484      0.007     19.942      0.000       0.134       0.163
number_of_reviews_ltm           

**🤓  ¿Que podemos ver en summary? 👇🏻**


**El modelo ha empeorado al eliminar "room_type"  y "minimum_nights" , no ha mejorado la situación**

**R-squared: es igual a 0.041 quiere decir que este modelo de precicción cumple su cometido en un 4,2% un número muy bajo**